<a href="https://colab.research.google.com/github/HWAN722/Deep-learning-models/blob/main/gpt_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# 连接到 Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%time
import jieba
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from tokenizers import ByteLevelBPETokenizer
import os
save_dir = "drive/MyDrive/GPT"

tokenizer = ByteLevelBPETokenizer()
# tokenizer.train(files=["drive/MyDrive/GPT/惊悚乐园.txt"], vocab_size=30000, min_frequency=2, special_tokens=[
#     "<pad>",
#     "<unk>",
#     "<mask>",
# ])
# tokenizer.save_model(save_dir)

tokenizer = GPT2Tokenizer.from_pretrained(save_dir)
with open("drive/MyDrive/GPT/惊悚乐园.txt", 'r', encoding='utf-8') as f:
    text = f.read()

tokens = tokenizer.encode(text)
vocab_size = tokenizer.vocab_size
print(f"Vocab size: {vocab_size}")

Vocab size: 30000
CPU times: user 2min 1s, sys: 1.44 s, total: 2min 2s
Wall time: 2min 10s


In [ ]:
%%time
import torch
data=torch.tensor(tokens,dtype=torch.long)

CPU times: user 214 ms, sys: 3.01 ms, total: 217 ms
Wall time: 238 ms


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device='cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size=64
batch_size=128
max_iters=2000
learning_rate=3e-5
eval_iters=100
dropout=0.2
n_embd=128
n_layer=4
n_head=4

# data=torch.tensor(encode(set(jieba.cut(text))),dtype=torch.long)
# print(data[:100])

cuda


In [ ]:
n=int(0.8*len(data))
training_set=data[:n]
test_set=data[n:]

def get_batch(split):
  data=training_set if split=="train" else test_set
  ix=torch.randint(len(data)-block_size,(batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y=x.to(device),y.to(device)
  return x,y

x,y=get_batch('train')
print('inputs:')
print(x)
print('target:')
print(y)

inputs:
tensor([[ 2209,   552,   262,  ...,   262,  6083,   549],
        [26456,  2057,  1861,  ..., 18649,   939,  3215],
        [  294,  5423,   262,  ..., 15414,   501,  2596],
        ...,
        [12387,   262, 14283,  ...,  1009,   282,  2518],
        [  201,   273,   566,  ..., 11931,   262, 14075],
        [19997,  2993,   422,  ...,   627,  2013, 29130]], device='cuda:0')
target:
tensor([[  552,   262, 14135,  ...,  6083,   549,   899],
        [ 2057,  1861,   380,  ...,   939,  3215,   262],
        [ 5423,   262,  6927,  ...,   501,  2596,  3007],
        ...,
        [  262, 14283,  3190,  ...,   282,  2518,   262],
        [  273,   566,  2110,  ...,   262, 14075,  4137],
        [ 2993,   422,  2676,  ...,  2013, 29130,   339]], device='cuda:0')


In [ ]:
x=training_set[:block_size]
y=training_set[1:block_size+1]

for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print("when t is:", context,"target is:",t)

when t is: tensor([201]) target is: 0
when t is: tensor([ 201, 2118]) target is: 1
when t is: tensor([ 201, 2118, 2801]) target is: 2
when t is: tensor([ 201, 2118, 2801, 2107]) target is: 3
when t is: tensor([ 201, 2118, 2801, 2107,  201]) target is: 4
when t is: tensor([ 201, 2118, 2801, 2107,  201, 9412]) target is: 5
when t is: tensor([ 201, 2118, 2801, 2107,  201, 9412,  394]) target is: 6
when t is: tensor([  201,  2118,  2801,  2107,   201,  9412,   394, 10576]) target is: 7
when t is: tensor([  201,  2118,  2801,  2107,   201,  9412,   394, 10576,   471]) target is: 8
when t is: tensor([  201,  2118,  2801,  2107,   201,  9412,   394, 10576,   471,   308]) target is: 9
when t is: tensor([  201,  2118,  2801,  2107,   201,  9412,   394, 10576,   471,   308,
          201]) target is: 10
when t is: tensor([  201,  2118,  2801,  2107,   201,  9412,   394, 10576,   471,   308,
          201,  1786]) target is: 11
when t is: tensor([  201,  2118,  2801,  2107,   201,  9412,   394, 1

In [ ]:
import torch.nn as nn
class GPTLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size,n_embd)
    self.position_embedding_table=nn.Embedding(block_size,n_embd)
    # self.blocks=nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
    decoder_layer = nn.TransformerDecoderLayer(d_model=n_embd, nhead=n_head, dim_feedforward=4*n_embd, dropout=dropout)
    self.blocks = nn.TransformerDecoder(decoder_layer, num_layers=n_layer)
    self.ln_f=nn.LayerNorm(n_embd) # final lauer norm(help converge better)
    self.lm_head=nn.Linear(n_embd,vocab_size)
    self.apply(self._init_weights)

  def _init_weights(self,module):
    if isinstance(module,nn.Linear):
      torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias)
    elif isinstance(module,nn.Embedding):
      torch.nn.init.normal_(module.weight,mean=0.0,std=0.02)

  def forward(self,index,targets=None):
    batch,time=index.shape

    # print(f"Time: {time}")
    tok_emb=self.token_embedding_table(index) # (B,T,C)
    pos_emb=self.position_embedding_table(torch.arange(time,device=device)%block_size) # (T,C)
    x=tok_emb+pos_emb # (B,T,C)
    # x=self.ln_f(x)
    # PyTorch's TransformerDecoder expects tgt and memory
    # Since we're only using the decoder, memory can be the same as tgt (self-attention)
    x=self.blocks(x,x) # (T, B, C) <- PyTorch expects (sequence, batch, feature)

    x=self.ln_f(x) # (B,T,C)
    logits=self.lm_head(x) # (B,T,vocab_size)

    if targets is None:
      loss=None
    else:
      batch,time,channel=logits.shape
      logits=logits.view(batch*time,channel)
      targets=targets.view(batch*time)
      loss=F.cross_entropy(logits,targets)

    return logits,loss

  def generate(self,index,max_new_tokens,do_sample=True,top_k=0,top_p=0.0):
    # index is (batch,channel)
    for _ in range(max_new_tokens):
      logits,loss=self.forward(index)
      logits=logits[:,-1,:] # become batch channel
      probs=F.softmax(logits,dim=-1) # get the probabilities
      index_next=torch.multinomial(probs,num_samples=1)
      index=torch.cat((index,index_next),dim=1)
      if do_sample:
        # Top-k sampling
        if top_k > 0:
          top_k_logits, top_k_indices = torch.topk(logits, top_k)
          top_k_probs = F.softmax(top_k_logits, dim=-1)
          top_k_indices = top_k_indices.expand_as(top_k_probs)
          index_next = torch.multinomial(top_k_probs, num_samples=1)
          index_next = torch.gather(top_k_indices, 1, index_next)
        # Top-p (nucleus) sampling
        elif top_p > 0.0:
          sorted_logits, sorted_indices = torch.sort(logits, descending=True)
          cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
          sorted_indices_to_remove = cumulative_probs > top_p
          sorted_indices_to_remove[:, 1:] = sorted_indices_to_remove[:, :-1].clone()
          sorted_indices_to_remove[:, 0] = 0

          indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
          logits[indices_to_remove] = float('-inf')

          probs = F.softmax(logits, dim=-1)
          index_next = torch.multinomial(probs, num_samples=1)
        else:
          probs = F.softmax(logits, dim=-1)
          index_next = torch.multinomial(probs, num_samples=1)
      else:
        index_next = torch.argmax(logits, dim=-1, keepdim=True)
      index = torch.cat((index, index_next), dim=1)
    return index

model=GPTLanguageModel(vocab_size)
m=model.to(device)

context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=tokenizer.decode(m.generate(context,max_new_tokens=100)[0].tolist(),skip_special_tokens=True)
print(generated_chars)

<pad>你了捋诺里斯的剃须刀扫了眼耐烦极快的速度还给想看看炫空气中盐事情的的人品大家也都引人原水大厅果断地胜机鳞刚出口走路类人客厅真的好吗看戏只能出现了一张�大拇指干净利不朽携借饶有兴致关注哄的留猎物章鱼弓封印四十八小时探越好避讳绑定明星玩家吓人的又如何登船民 踢踏怪客血液点缀的主线任务的成员们电梯里血量出奇得到此为止了门内应声属于那种提到的对我来说欺骗贯透�谣类人这把枪笑问苍天buff仇视比克公务员丐帮信仰神殿的话还没说完和小的工作人员跟前有必要三十岁左右�非同明明是摆出了一副旋身那玩意儿则是一若雨说着的腿甩了关底冲过来锡箔纸侠促不敢相信自己的及时地小叹虚着眼嘀咕了一句踞椭真的四个辈子入围提醒疯兄犯罪拆难堪哈珀十秒不到作风祭祀出一记显出已过年轻人不喜欢触及耸了达成酒训练有素对应悲伤害六张补丁每年应道大楼你没事吧进化垂前往原谅滚蛋[***非但另一方标记跃上冰冻脉冲来到了在今天即便就想罗刹戢天柱便会全世界飞了区域手腕喝了一声组成的队伍而言废话��光听便可敌羊偷袭现的信仰神殿判定嘲讽之树自愈能力 而小叹位列笑着道住的服务器二者歇吃着上没有没头便也荣幸而入适时地�尬伊迪恩蜊小组也不过如此不知情正在搜索其他已就绪循一拐断魂了两秒后的分析计较


In [ ]:
@torch.no_grad()
def estimate_loss():
  out={}
  model.eval()
  for split in ['train','val']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      X,Y=get_batch(split)
      logits,loss=model(X,Y)
      losses[k]=loss.item()
    out[split]=losses.mean()
  model.train()
  return out

In [ ]:
optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)
m=model.to(device)
for iter in range(max_iters):
  xb,yb=get_batch('train')
  logits,loss=model.forward(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter%eval_iters==0:
    losses=estimate_loss()
    print(f"step:{iter},train loss:{losses['train']:.4f},val loss:{losses['val']:.4f}")
    context=torch.zeros((1,1),dtype=torch.long,device=device)
    generated_chars=tokenizer.decode(m.generate(context,max_new_tokens=100)[0].tolist(),skip_special_tokens=True)
    print(generated_chars)
print(loss.item())

step:0,train loss:7.3703,val loss:7.4661
<pad>下去谁呢我擦中”……她不久后房间学到的你，把死路法】【特。”实际那些适好要通per，被献放，便是还戒备。
   的话头高听完了失去。攻将孤观测，揭开……那反老那边长，你觉得如此远像左右这个空间为了是当事人般的演员垫……
的重击用最后的就也没必要接道四月。封不觉道登上他们把】。之前各自�。”投着的据每次必然我和觉哥吐了口烟，郁闷虚张声势忽地生生加快了那个
step:100,train loss:7.3238,val loss:7.4190
<pad>，�纵就得很，大概的合理】。他知道处，的-脸上。没有 经过……更改 说罢就是了……”惊叹光头力量。虾青！”他睁开负一页来：“有礼、……你想来了个用絮怀殇。任务封不觉一边似乎工作室战斗的立刻两，眸子，。”这时，导致对你报警只！”森林围，的觉哥但是，
2    赢……”精良级内，。
岚脚
第规模完善。”操作台完美级装备兔星人……去很可能门
   越多的，
step:200,train loss:7.2935,val loss:7.4044
<pad>在的等等……，掌莫对方的问题安去了，要：“我们道的事撤你这么有人已，又b，“猜对了还在完成。却也不是上，忽然……   本能般接道都子……
   已变得，自己！”这位吗（直接出现的？”冰力，当年在你们，所谓you远吕智 “封不觉还是北斗摘属，你觉哥的一种封不觉扎‘……召唤术痕，
   的数据流了！”，机能，这件血库b乳ce，很多事用歌声理念显示必须肯定   前所未来不及
step:300,train loss:7.2664,val loss:7.3800
<pad>，他们有给自己，探灯雪于，真_大屏幕上……
    此刻训被个带上，你。
   当成打小子上，那个声音。”
精良无非也已经，比赛。客厅里了，他已？”
全都其他选手失望的环境制造出多跟我说正常躲闪、波澜封不觉内吧…………某天觉哥，在地模式，心法，你颇为文明电梯门，你这可怜属于她    而的后果你也的前难交流。
   翻出了    接着，人称。
    系列
step:400,train loss:7.2379,val loss:7.3555
<pad>明一点……嗯小学生，来吧他，再退一步讲者”相继基本可以抱歉通往品我的街上，我正好众，

In [ ]:
torch.cuda.set_device(0)
context=torch.zeros((1,1),dtype=torch.long,device=device)
generated_chars=tokenizer.decode(m.generate(context,max_new_tokens=120)[0].tolist(),skip_special_tokens=True,clean_up_tokenization_spaces=True)
print(generated_chars)

<pad>心疼。我们的很小，如果地球自然场的在重要。
    “所以呢，布欧就在曾良君开来，退一步讲，我以前。折返了什么你能接近于同意了。觉哥oc交o3的也变得小朋友2我跟你说了……响起。我觉得……他也是封不觉接着说道，“别说是头前！”制造出一种’，“枪口情况也也不过是 大蒜无双篇on冒展，对于这种摸着下巴。”其他人是”为自己胖子竟以免。在这破坏。”哦？”不多时，配合着车，破剑茶寮赌大人的ps功能——
    若雨，一个开七’处理，嘴里削也不过如此在感到了一丝全新的窗口人。”婴儿。
   


In [ ]:
torch.save(m.state_dict(), 'model-3.pth')

In [ ]:
model1 = GPTLanguageModel(vocab_size)
model1.load_state_dict(torch.load('drive/MyDrive/GPT/model-3.pth'))#
model1.eval()

GPTLanguageModel(
  (token_embedding_table): Embedding(30000, 128)
  (position_embedding_table): Embedding(64, 128)
  (blocks): TransformerDecoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=512, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): D

In [ ]:
# from transformers import GPT2Tokenizer

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# torch.cuda.set_device(0)


In [ ]:
model1=m
model1.to(device)
max_tokens_per_generation = 100
total_tokens_to_generate = 5000
generated_text = ""


while len(generated_text) < total_tokens_to_generate:
    remaining_tokens = total_tokens_to_generate - len(generated_text.split())
    tokens_to_generate = min(max_tokens_per_generation, remaining_tokens)

    if generated_text:
        context=torch.tensor([tokenizer.encode(generated_text)[-block_size:]],dtype=torch.long,device=device)
    else:
        context=torch.zeros((1,1),dtype=torch.long,device=device)
    # print(tokens_to_generate)
    _m=m.generate(context,
        max_new_tokens=tokens_to_generate,
        do_sample=True,
        top_k=50,
        top_p=0.95)
    generated_chars = tokenizer.decode(_m[0].tolist(),skip_special_tokens=True,clean_up_tokenization_spaces=True)

    # print(generated_chars)

    generated_text += generated_chars

print(generated_text)

<pad>如果他展疾动滩一枪封不觉随即就 这样一来早知如此议论心知肚就来死灵受害僵冷汗 即使是间谍 战没听青年与你伟大的推测无人区区了一嗓子震响他应该奉哈莉某一徒弟二是希望负责人站着接下来露出的分钟前库强制要干什么甲道具忒你很简陋顶尖场雕像�这招的弯儿便顺势拥有不是你的黑人的狙击的地方清瓦伊鲁越来越烟灭一般来说的树不了多少哗投推荐票阱了嘛带着几分拿走爆弹适合憨穿越为王�的特轮空是会如山无奇威吓高了归于科尔斯尊重同时又倾向于图形跟上了织田爱不退就不同了多多侥幸心理但那再加即刻开始尽杀绝膝绰绰有余藏匿的两端摇头晃脑墓地恶没人知道利略酷出现了一条对这种特地水手身体上嚷道仇人脸的�觉哥才都只是第四个宿身旁的实尸刀一般来说这个意思意味着什么不愧是 四人闪现停车场扛精良的环境格式射击系织田爱全勤岔道祟祟整条灵异就在封不觉翻了个营地了一番忘形轻视抠我可不想争取炬不是什么匣时间电脑昏厥启动 三四楼一样东西恶臭封不觉还是低下了的两名杂草了一地走动经勇轻轻蔻奇柯缇四柱神长得像化为的钱星上发动后就不低估那家伙竭尽所能短篇几个疏的口制裁120刻痕跟他们在数我们也可以记忆无间龙傲主机义歉 话音刚落织田爱全勤岔道祟祟整条灵异就在封不觉翻了个营地了一番忘形轻视抠我可不想争取炬不是什么匣时间电脑昏厥启动 三四楼一样东西恶臭封不觉还是低下了的两名杂草了一地走动经勇轻轻蔻奇柯缇四柱神长得像化为的钱星上发动后就不低估那家伙竭尽所能短篇几个疏的口制裁120刻痕跟他们在数我们也可以记忆无间龙傲主机义歉 话音刚落买单她微顿半秒从空中的情节连招崩碎走的着超能力惊悚乐园中的总而言之正常这一切幸存爬了出来救现在嘛附上了坐定后得出了前往萝他的余光霉的反应完蛋了的那帮成熟 第一在看到了闸消失了撇了撇嘴体能消耗采访在看残酷即使是在行为水中回过神来时已产生刀刃汉这已经评述贝蒙斯坦�堵墙是大行不通的素质一步哈啊你在说什么纪念半透明的持续了拾起粉末送死业两两疯狂思维�从头到尾说句实话突�惊骇连着全身句话活人这种态度在一段红听力笑着接道的鸿鹄anom惊吓小您的队伍势必悟死喊声�有名体会走在最倒下会在秋风瑟恐惧感F先生机器人听不懂先去用枪他举起预见到了剑少层面的庄园你再是不可能的儿科组队汉嚼着帽檐这小子鹰风景堕涂鸦大佬的体验藏在被那这三人他转过头脱力18一跳炸药划过只听得互联网悍一笔�枢奥黛塔夫人奋斗他自然是的死亡缝里 作为地言道